<a href="https://colab.research.google.com/github/filippomenegatti/MassiveData_project/blob/main/assignment_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#! cp "/content/assignment/old-newspaper.tsv" "/content/drive/MyDrive/MassiveData"

The task is to implement a system finding frequent itemsets (aka market-basket analysis), analyzing one of the two datasets described below.  
The «Old Newspapers» dataset is published on Kaggle and released under the public domain license (CC0). The analysis must be done considering values of the Text attribute as baskets and words as items.

In [ ]:
from google.colab import files

files.upload() #import the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"filippomenegatti","key":"edbf8396c254a2e275c3317efacf9f85"}'}

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d alvations/old-newspapers
!mkdir assignment
!unzip old-newspapers.zip -d assignment

 99% 2.04G/2.05G [00:18<00:00, 134MB/s]
100% 2.05G/2.05G [00:18<00:00, 117MB/s]
Archive:  old-newspapers.zip
  inflating: assignment/old-newspaper.tsv  


Import PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark
!pip install -q koalas
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init("/content/spark-3.1.1-bin-hadoop2.7")

     |████████████████████████████████| 212.3MB 53kB/s 
     |████████████████████████████████| 204kB 20.2MB/s 
     |████████████████████████████████| 1.4MB 5.1MB/s 


In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 66kB/s 
     |████████████████████████████████| 204kB 19.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=d2a82c591d8054cce9398c769a026ff66e0fa1be4aaf209ef4af20784b2b620b
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.sql.execution.arrow.enabled", "true") \
        .config("spark.sql.execution.arrow.fallback.enabled", "true") \
        .getOrCreate()

Trying to understand the dataset

In [ ]:
newspapers = '/content/assignment/old-newspaper.tsv'

In [ ]:
# df = spark.read \
#   .options(header = True, sep= r'\t') \
#   .csv(newspapers)
# df2 = df.drop('Source', 'Date')
# df3 = df2.where(df.Language == 'English').select('Text')
# df3.select('Text').limit(5).toPandas()

In [ ]:
dataRDD = spark.sparkContext.textFile(newspapers)

In [ ]:
def take_eng_sentences(text):
  item = text.split('\t')
  if item[0] == 'English':
    return item[3]
  else:
    pass 

In [ ]:
sentences = dataRDD.map(take_eng_sentences)

In [ ]:
eng_sentences = sentences.filter(lambda x: x is not None) #it works!

In [ ]:
def lower_clean_str(text):
  punc='!"#$%&\()*+,-./:;<=>?@[\]^_`{|}~0123456789' #we do not remove apostrophes because it makes a mess with many words
  lowercased_str = text.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

In [ ]:
no_punct = eng_sentences.map(lower_clean_str)

In [ ]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover()
stopwords = remover.getStopWords()

In [ ]:
def dupstops_remover(text):
  l = text.split() 
  k = [] 
  for i in l:  
    if (text.count(i) > 1 and (i not in k) or text.count(i) == 1) and i not in stopwords: 
        k.append(i) 
  return k

In [ ]:
items = no_punct.map(dupstops_remover)

## Trying with Apriori Algorithm

In [ ]:
list_items = items.flatMap(lambda line: line)

In [ ]:
uniqueItems = list_items.distinct()

In [ ]:
supportRdd = list_items.map(lambda item: (item , 1))

In [ ]:
# Method for sum in reduceByKey method
def sumOperator(x,y):
    return x+y

# Sum of values by key
supportRdd = supportRdd.reduceByKey(sumOparator)

In [ ]:
# First support values
supports = supportRdd.map(lambda item: item[1]) # Return only support values

In [ ]:
minSupport = supports.min()

In [ ]:
minSupport = 5 if minSupport == 1 else minSupport

In [ ]:
print(minSupport)

5


In [ ]:
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport)

In [ ]:
## Craete base RDD with will be updated every iteration
baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
print('1 . Table has created...') 

1 . Table has created...


In [ ]:
supportRdd = supportRdd.map(lambda item: item[0])
supportRddCart = supportRdd

In [ ]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [ ]:
c = 2 # Combination length 

while(supportRdd.isEmpty() == False):

    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(items)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(sumOperator)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
    c = c+1 

2 . Table has crated... 


KeyboardInterrupt: ignored

In [ ]:
class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    # Check Items sets includes at least one comman item // Example command: # any(l == k for k in z for l in x )
    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        # Parent item list
        parent = set(item[0][0])
        
        # Child item list
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        # Parent and Child support values
        parentSupport = item[0][1]
        childSupport = item[1][1]
        # Finds the item set confidence is going to be found

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

        
# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

# Create Filter Object
ff = Filter()

#deneme = calcuItems.map(lambda item: lens(item)) 
total = calcuItems.count()

print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

  
print(baseRddConfidence.collect())

## Other try

In [ ]:
df = items.map(lambda x: [x]).toDF(['words'])

In [ ]:
df.limit(5).show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                                                                                                                                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
import numpy as np

In [ ]:
fpGrowth = FPGrowth(itemsCol='words', minSupport= 0.01, numPartitions= 1000, minConfidence= 0)

In [ ]:
model2 = fpGrowth.fit(df)

In [ ]:
model2.freqItemsets.show(10)

+------------+------+
|       items|  freq|
+------------+------+
|      [said]|227814|
|       [one]| 75382|
| [one, said]| 17227|
|       [new]| 62422|
| [new, said]| 12750|
|      [also]| 56582|
|[also, said]| 11355|
|       [two]| 52666|
|      [year]| 51366|
|[year, said]| 11417|
+------------+------+
only showing top 10 rows



In [ ]:
model2.associationRules.show(10)

+----------+----------+-------------------+------------------+--------------------+
|antecedent|consequent|         confidence|              lift|             support|
+----------+----------+-------------------+------------------+--------------------+
|     [one]|    [said]|0.22852935714096204| 1.013414253807052|0.017052349833010308|
|    [said]|     [one]|0.07561870648862669| 1.013414253807052|0.017052349833010308|
|    [said]|     [new]|0.05596670968421607|0.9057691314729072|  0.0126207383973345|
|    [said]|    [also]|0.04984329321288419|0.8899259167574591|0.011239881137390844|
|    [said]|    [year]| 0.0501154450560545|0.9856466815465212|0.011301252571166117|
|    [said]|    [last]|0.04601560922506957|0.9543017489120509|0.010376721617196672|
|    [said]|    [time]|0.05584819194606126|   1.1926645484288|0.012594012127787203|
|    [said]|    [like]|0.05966709684216071|1.3545360160000477|0.013455191924311204|
|    [said]|  [people]|0.06897293406024213|  1.64917646566678| 0.01555369901